In [21]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

BASE: str = 'https://www.metacritic.com'
API: str = 'browse/movies/score/metascore/all/filtered'
QUERY = lambda n: f'sort=desc&page={n}'

AGENT: str = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'

TO_PAGE: int = 148
DATA_PATH: str = '../data/metacritic'

In [22]:
def scrape_page(num: int):
    url: str = f'{BASE}/{API}?{QUERY(num)}'

    s = requests.Session()
    s.headers['User-Agent'] = AGENT

    page = s.get(url, allow_redirects=True)
    soup = BeautifulSoup(page.content, "html.parser")

    return pd.DataFrame.from_records([
        {
            'title': movie.find_all('h3')[0].text,
            'metascore': movie.find_all('div', class_=['movie'])[0].text,
            'userscore': movie.find_all('div', class_=['user'])[0].text,
            'summary': movie.find_all('div', class_='summary')[0].text
        }
        for movie in soup.find_all("tr") if movie.get('class', [None])[0] != 'spacer'
    ])

In [23]:
scraped = pd.concat(
    [scrape_page(n) for n in range(TO_PAGE + 1)],
    ignore_index=True
)
scraped.head(32)

,title,metascore,userscore,summary
0,The Godfather,100,9.2,\n Francis Ford Coppola...
1,Citizen Kane,100,8.4,\n Following the death ...
2,Rear Window,100,8.7,\n A wheelchair-bound p...
3,Casablanca,100,8.8,"\n A Casablanca, Morocc..."
4,Boyhood,100,7.4,\n Filmed over 12 years...
5,Three Colors: Red,100,8.5,\n Krzysztof Kieslowski...
6,Vertigo,100,8.6,\n Vertigo creates a di...
7,Notorious,100,7.9,\n A woman is asked to ...
8,Singin' in the Rain,99,8.7,\n A silent film produc...
9,City Lights,99,8.9,\n The Tramp (Charlie C...


In [25]:
scraped.to_csv(f'{DATA_PATH}._raw.csv', index=False)